# Instacart Market Basket Analysis

# Introduction

For reference, the information is sourced from https://www.kaggle.com/c/instacart-market-basket-analysis/overview and the data dictionary is sourced from https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b

“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on <date>

Goal: To predict the product/s in an order

The five important steps involved in Data Science is as shown below:

1. Getting the data.
2. Cleaning the data
3. Exploring the data
4. Building the data
5. Presenting the data

# Initialization

In [ ]:
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
color = sns.color_palette()

In [ ]:
df_aisles = pl.read_csv('../data/raw/aisles.csv', low_memory = False)
df_aisles.head()

In [ ]:
df_departments = pl.read_csv('../data/raw/departments.csv', low_memory = False)
df_departments.head()

In [ ]:
df_products = pl.read_csv('../data/raw/products.csv', low_memory = False)
df_products.head()

At first glance, the products table can be denormalized and joined with the aisles and departments tables.

In [ ]:
df_orders = pl.read_csv('../data/raw/orders.csv', low_memory = False)
df_orders.head()

This table indicates the user ID and order number which will help associate users to an order and understand the product mix for prior and subsequent orders. More importantly, the order set is specified which categorizes orders into prior orders, training orders, and testing orders. Time series analysis can also be performed with the date of week, hour of day, and days since prior order.

# Exploration

In [ ]:
df_orders.shape

In [ ]:
df_orders.describe()

There are over 3 million rows in the orders table, and describe shows most reorders occur 1.5 to 7 days after their prior order.

In [ ]:
df_orders['eval_set'].unique()

The orders data has been conveniently labeled with prior, test, and train to help our analysis.

## Order Count

In [ ]:
df_order_count = (
    df_orders
    .groupby('user_id')
    .agg(
        [
            pl.col('user_id').count()
        ]
    )
    .sort('user_id_count', reverse=True)
)

df_order_count.head()

In [ ]:
df_order_count.shape

In [ ]:
fig = sns.histplot(x=df_order_count['user_id_count'])
fig.show()

The histogram of the order number shows that the orders are not uniformly distributed as expected with a heavy right skew, and an interesting spike at what appears to be at 100 orders.  Let's determine the size of 80% of our orders.

In [ ]:
fig = sns.histplot(x=df_order_count['user_id_count'], stat='probability', cumulative=True)

fig.show()

At a quick glance, the orders table shows that roughly 80% of users have ordered from 4 to 25 times and it may be prudent to limit our data to this subset to efficiently model our predictions.  Note that orders with less than 4 items were not included in the original dataset.

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=df_orders.to_pandas(), x='order_dow', color=color[0])

plt.ylabel('Order count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)

plt.title('Frequency of order by day of week', fontsize=15)

plt.show()

In [ ]:
fig = sns.histplot(x=df_orders['order_dow'], stat='percent')
fig.show()

The data dictionary does not explicity state which days these numbers correspond to so it will be presumed that 0 and 1 correspond to Saturday and Sunday.  It is plausible that users may have orders on the weekend to prepare for the week ahead.

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=df_orders.to_pandas(), x='order_hour_of_day', color=color[0])

plt.ylabel('Order count', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)

plt.title('Frequency of order by hour of day', fontsize=15)

plt.show()

In [ ]:
fig = sns.histplot(x=df_orders['order_hour_of_day'], stat='percent')
fig.show()

As expected, most orders occur during daytime hours.

In [ ]:
df_orders_group_by_dow_hod = (
    df_orders
    .groupby(['order_dow', 'order_hour_of_day'])
    .agg(
        [
            (pl.col("order_number").count().alias("order_count_by_dow_hod"))
        ]
        )
)

df_orders_pivot = df_orders_group_by_dow_hod.pivot(index='order_dow', column='order_hour_of_day', values='order_number')

# plt.figure(figsize=(12,6))
# sns.heatmap(df_orders_pivot)
# plt.title('Frequency of day of week versus hour of day')

# plt.show()

It would appear that Sunday mornings from 9-11am and Saturday afternoons from 1-3pm are the most popular times of day for orders.

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.countplot(data=df_orders.to_pandas(), x='days_since_prior_order', color=color[3])

plt.ylabel('Order count', fontsize=12)
plt.xlabel('Days since prior order', fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Frequency distribution by days since prior order', fontsize=15)

plt.show()

In [ ]:
fig = sns.histplot(x=df_orders['days_since_prior_order'], stat='percent')
fig.show()

Both 30 and 7 days stand out as the most common number of days since a user's prior order.

# Data Preparation

In [ ]:
df_train_order_products = pl.read_csv('../data/raw/order_products__train.csv', low_memory = False)
df_train_order_products.head()

In the training data set, the products in the orders are shown.  The cart order sequence and the reordered flag will undoubtedly be insightful.

In [ ]:
df_order_products_prior = pl.read_csv('../data/raw/order_products__prior.csv', low_memory = False)
df_order_products_prior.head()

There is another dataset for the prior orders for products.  Let's check if these records are distinct or included within the training set. 

In [ ]:
df_order_products_combined = pl.concat([df_train_order_products, df_order_products_prior])

df_order_products_combined.shape

In [ ]:
df_order_products_combined.drop_duplicates().shape

With this we have validated that the records are distinct from each other and can use this to understand the whole picture after joining with our remaining datasets.

In [ ]:
df_products_aisles = df_products.join(df_aisles, on='aisle_id', how='left')
df_products_aisles_departments = df_products_aisles.join(df_departments, on='department_id', how='left')

df_products_aisles_departments.head()

In [ ]:
df_order_products_combined = df_order_products_combined.join(df_products_aisles_departments, on='product_id', how='left')

df_order_products_combined.head()

Notice that the orders are not linked with the user.

In [ ]:
df_train_order_products_combined = df_order_products_combined.join(df_orders, on='order_id', how='left')

df_train_order_products_combined.head()

This table brings over 32 million orders and mirrors the structure of the training set.  The left join is used to filter out the testing data for orders.



In [ ]:
df_train_order_products_combined['eval_set'].unique()

The data frame only contains the prior and train data, and we can now output this as a CSV for model building.

In [ ]:
# df_train_order_products_combined.to_pickle('../data/processed/data_order_products.pkl')

In [ ]:
product_count = df_train_order_products_combined['product_name'].value_counts().head(10)
product_count.columns = ['product_name', 'frequency_count']

product_count

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=product_count.to_pandas(), x='frequency_count', y='product_name', alpha=0.8)
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Aisle', fontsize=12)

plt.show()

In [ ]:
fig = sns.histplot(x=product_count['frequency_count'], cumulative=True)

fig.show()

Organic fruits and vegetables immediately stand out, so let's dive into the aisle.

In [ ]:
order_products_aisle = df_train_order_products_combined['aisle'].value_counts().head(10)
order_products_aisle.columns = ['aisle_name', 'frequency_count']

order_products_aisle

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=order_products_aisle.to_pandas(), y='aisle_name', x='frequency_count', alpha=0.8)
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Aisle', fontsize=12)

plt.show()

Let's understand how large the orders are

In [ ]:
df_product_count = (
    df_order_products_combined
    .groupby('order_id')
    .agg(
        [
            pl.col('order_id').count()
        ]
    )
    .sort('order_id_count', reverse=True)
)

df_product_count.head()

In [ ]:
fig = sns.histplot(x=df_product_count['order_id_count'], stat='probability', cumulative=True)

fig.show()

For the actual orders, we can spot that roughly 15 items are in the cart for 80% of the orders. 

In [ ]:
df_train_order_products_combined['reordered'].sum() / df_train_order_products_combined.shape[0]

In the entire data set, 59% of the orders were classified as reordered.

## TBD

Let's examine the order distribution for users.

In [ ]:
user_id_count =  (
    df_orders
    .groupby('user_id')
    .agg(
        [
            pl.col('user_id').count()
        ]
    )
    .sort('user_id_count', reverse=True)
)

user_id_count.head()

There are over 200,000 users in this dataset.

In [ ]:
fig = sns.histplot(x=user_id_count['user_id_count'], stat='probability', cumulative=True)

fig.show()

For efficiency, it is reasonable to look only at users with 20 orders maximum as it comprises almost 80% of the data.

In [ ]:
fig = sns.histplot(x=user_id_count['user_id_count'], binrange=[4, 20])

fig.show()

In [ ]:
order_id_count =  (
    df_train_order_products
    .groupby('order_id')
    .agg(
        [
            pl.col('order_id').count()
        ]
    )
    .sort('order_id_count', reverse=True)
)

order_id_count.head()

In [ ]:
fig = sns.histplot(x=order_id_count['order_id_count'], stat='probability', cumulative=True)

fig.show()

For efficiency, it is reasonable to look at orders with 20 products maximum as it comprises almost 90% of the data.

In [ ]:
order_id_prior_count =  (
    df_order_products_prior
    .groupby('order_id')
    .agg(
        [
            pl.col('order_id').count()
        ]
    )
    .sort('order_id_count', reverse=True)
)

order_id_prior_count.head()

In [ ]:
fig = sns.histplot(x=order_id_prior_count['order_id_count'], stat='probability', cumulative=True)

fig.show()

The prior orders also validate the previous point of looking at orders with 20 products maximum.

In [ ]:
df_train_order_products_named = df_train_order_products.join(df_products, on='product_id', how='inner')

df_train_order_products_named.head()

The orders table does not have the product name due to normalization so we denormalize it to bring the product name back into play

In [ ]:
products_value_counts = df_train_order_products_named['product_name'].astype('category').value_counts()

products_value_counts.head(10)

A quick glance into the data highlights the popularity of organic goods

In [ ]:
df_train_order_products['product_id'].astype('category').value_counts()[:2].plot.hist(cumulative = True, density = True, figsize = (12, 6))

It may be worth to examine to apply the pareto principle to the original 40,000 products to determine potentially which 20% of unique products result in 80% of the orders.

In [ ]:
df_order_products_prior_named = pl.merge(df_order_products_prior, df_products, on='product_id', how='inner')

df_order_products_prior_named.head()

Let's take a look at the most popular products in the prior orders, regardless of whether they were or were not reordered

In [ ]:
products_prior_value_counts = df_order_products_prior_named['product_name'].astype('category').value_counts()

products_prior_value_counts.head(10)

Many of the current popular products are unsurprisingly popular products for prior orders